In [ ]:
import random
from ipywidgets import Output, HBox
from ipycanvas import MultiCanvas
from IPython.display import Audio

In [ ]:
#Prüfung ob das Schiff auf das Spielfeld passt
def Prüfung_Schiff_in_Spielfeld(Schiffslaenge, Zeile, Spalte, Ausrichtung):
    '''Mit dieser Funktion wird überprüft, ob das Schiff auf das Feld passt oder nicht resp. ob sich das Schiff
    innerhalb des Spielfelds befindet.'''
    if Ausrichtung == 'W':
        if Spalte + Schiffslaenge > N:
            return False
        else:
            return True
    else:
        if Zeile + Schiffslaenge > N:
            return False
        else:
            return True

#Prüfung ob das Schiff mit einem anderen Schiff kolidiert
def Schiffsüberschneidung(Spielfeld, Zeile, Spalte, Ausrichtung, Schiffslänge):
    '''Mit dieser Funktion wird überprüft, ob die zu platzierenden Schiffe sich kreuzen resp. überschneiden. Dies wäre
    nicht legal für die Setzung der einzelnen Schiffe.'''
    if Ausrichtung == 'W':
        for x in range(Spalte, Spalte + Schiffslänge):
            if Spielfeld[Zeile][x] == '\u0298':
                return True
    else:
        for x in range(Zeile, Zeile + Schiffslänge):
            if Spielfeld[x][Spalte] == '\u0298':
                return True
    return False

#Platzierung der Schiffe durch den Spieler und den Computer
def Platzierung_Schiffe(Spielfeld):
    '''Mit dieser Funktion wird das Schiff entsprechend seiner Länge auf dem Spielfeld platziert'''
    for Schiffslänge in Schiffe:
        while True:
            #Hier wird geprüft, ob das zu platzierende Schiff auf das Spielfeld passt und nicht mit anderen Schiffen kolidiert
            Ausrichtung, Zeile, Spalte = random.choice(['W', 'S']), random.randint(0, N-1), random.randint(0, N-1)
            #Prüfung ob das Schiff ins Spielfeld passt
            if Prüfung_Schiff_in_Spielfeld(Schiffslänge, Zeile, Spalte, Ausrichtung):
                #Prüfung ob das Schiff kein anderes Schiff trifft
                if Schiffsüberschneidung(Spielfeld, Zeile, Spalte, Ausrichtung, Schiffslänge) == False:
                    #Platzierung des Schiffs
                    if Ausrichtung == 'W':
                        for x in range(Spalte, Spalte + Schiffslänge):
                            Spielfeld[Zeile][x] = '\u0298'
                    else:
                        for x in range(Zeile, Zeile + Schiffslänge):
                            Spielfeld[x][Spalte] = '\u0298'
                    break

def Platziere_Schiff(Spielfeld, Zeile, Spalte, Ausrichtung, Schiffslänge):
    if Prüfung_Schiff_in_Spielfeld(Schiffslänge, Zeile, Spalte, Ausrichtung):
        #Prüfung ob das Schiff kein anderes Schiff trifft
        if Schiffsüberschneidung(Spielfeld, Zeile, Spalte, Ausrichtung, Schiffslänge) == False:
            if Ausrichtung == 'W':
                for x in range(Spalte, Spalte + Schiffslänge):
                    Spielfeld[Zeile][x] = '\u0298'
            else:
                for x in range(Zeile, Zeile + Schiffslänge):
                    Spielfeld[x][Spalte] = '\u0298'
            return True

In [ ]:
# Output-Widget fuer Fehlermeldungen und Mitteilungen
log = Output(layout = {'border': '1px solid black'})

# x,y-Koordinaten in Zeilen, Spalten umrechnen  und umgekehrt
def rowcol2xy(row, col):
    y = (row+0.5) * SIZE/N
    x = (col+0.5) * SIZE/N
    return x, y
    
def xy2rowcol(x, y):
    col = int(x // (SIZE/N))
    row = int(y // (SIZE/N))
    return row, col

# Callback fuer Leinwand 2: schiesse auf geklicktes Feld
@log.capture()        
def on_mouse_down2(x, y):
    if mode == 'shoot':
        row, col = xy2rowcol(x, y)
        shoot((row, col))

# Callback fuer Leinwand 1:         
# Speichere geklicktes Feld in Attribut von fg2 (um globale Variable zu vermeiden)
@log.capture()        
def on_mouse_down1(x, y):
    row, col = xy2rowcol(x, y)
    fg2.selected_field = (row, col)        

# versuche Schiff zu platzieren
@log.capture()   
def on_mouse_up1(x, y):   
    src = fg2.selected_field
    if src is None or mode != 'place':
        return
    target = xy2rowcol(x, y)
    place(fg2.selected_field, target)   

In [ ]:
def draw_grid(canvas):
    for i in range(N+1):
        x = y = SIZE/N * i
        canvas.stroke_lines([(0, y), (SIZE, y)])
        canvas.stroke_lines([(x, 0), (x, SIZE)])

# Schiff = Kreis        
def draw_spielfeld(canvas, spielfeld):
    for row, cols in enumerate(spielfeld):
        for col, x in enumerate(cols):
            if not x.isspace():
                x, y = rowcol2xy(row, col)
                canvas.fill_circle(x, y, 8)
                
#Schiff zeichnen, die Farbe selber wird im unteren Teil des Codes definiert
def draw_ships(canvas, Zeile, Spalte, Ausrichtung, laenge):
    if Ausrichtung == 'S':
        for i in range(laenge):
            x, y = rowcol2xy(Zeile + i, Spalte )
            canvas.fill_circle(x, y, 8)                
    if Ausrichtung == 'W':
        for i in range(laenge):
            x, y = rowcol2xy(Zeile, Spalte + i)
            canvas.fill_circle(x, y, 8)       
            
def draw_ships(canvas, Zeile, Spalte, Ausrichtung, laenge):
    if Ausrichtung == 'S':
        for i in range(laenge):
            x, y = rowcol2xy(Zeile + i, Spalte )
            canvas.fill_circle(x, y, 8)                
    if Ausrichtung == 'W':
        for i in range(laenge):
            x, y = rowcol2xy(Zeile, Spalte + i)
            canvas.fill_circle(x, y, 8)       

#Schiff zeichnen, wenn geschossen wurde. Je nach Treffer oder Nicht-Treffer wechselt die Farbe
def shoot(target):
    log.clear_output()
    
    # Spieler schiesst
    fg2.fill_style = 'lightblue'
    fg2.fill_circle(*rowcol2xy(*target), 8)    
    if _shoot(0, target):
        fg2.fill_style = 'red'
        fg2.fill_circle(*rowcol2xy(*target), 10)

    # Computer schiesst
    target = random.randint(0, N-1), random.randint(0, N-1)
    bg1.fill_style = 'lightblue'
    bg1.fill_circle(*rowcol2xy(*target), 8)
    if _shoot(1, target):
        bg1.fill_style = 'red'
        bg1.fill_circle(*rowcol2xy(*target), 10)

#Funktion für die Schussabgabe. Dabei wird ein Leben abgezogen wenn getroffen wird und auch den Text-Output, falls gewonnen wird
def _shoot(player, target):
    '''gibt True zurueck, falls Treffer'''
    Zeile, Spalte = target
    hit =  Spielfelder[1-player][Zeile][Spalte] == '\u0298'    
    if hit:
        Leben[1-player] -= 1
        msg = '{}: {} hit a part of a ship! Remaining Player ship parts, Remaining Computer ship parts: {}'.format(target, players[player], Leben)
        if Leben[1-player] == 0:
            msg = '{} won. Congratulation'.format(players[player])
    else:
        msg = '{}: {} missed'.format(target, players[player])
        
    with log:
        print(msg)    
    return hit

#Platzierung der Schiffe
def place(src, target):
    global mode
    log.clear_output()
    
    if src[0] == target[0]:
        Ausrichtung = 'W'
        laenge = abs(target[1] - src[1]) + 1
    elif src[1] == target[1]:
        Ausrichtung = 'S'
        laenge = abs(target[0] - src[0]) + 1
    else:
        return
    
    Zeile = min(target[0], src[0])
    Spalte = min(target[1], src[1])
  
    if laenge in Schiffe and Platziere_Schiff(Spielfeld_Spieler, Zeile, Spalte, Ausrichtung, laenge):
        draw_ships(fg1, Zeile, Spalte, Ausrichtung, laenge)
        Schiffe.remove(laenge)
        msg = '{}-{}: The ships are placed on the board'.format(src, target)
        if not Schiffe:
            mode = 'shoot'
            msg += '\nSwitched to shooting mode! Try to shoot the ships from the Computer on the right board.'
    else: 
        msg = '{}-{}: The ship is too long, not on the board or not to place'.format(src, target)
        
    with log:
        print(msg)
        print('Ships to place: {}. Place them on the left board.'.format(Schiffe))                                    

In [ ]:
# 2 Leinwande a 2 Layers, bg, und fg
# werden in HBOx gepackt, um die nebeneinander anzuordnen
# links : Schiffe, rechts: Torpedos
SIZE = 200
layout={'border': '1px solid black'}
mcanvas1 = MultiCanvas(2, width=SIZE, height=SIZE, layout=layout)
mcanvas2 = MultiCanvas(2, width=SIZE, height=SIZE, layout=layout)

bg1, fg1 = mcanvas1
fg1.fill_style = 'green'

bg2, fg2 = mcanvas2
fg2.fill_style = 'red'
fg2.selected_field = None

mcanvas2.on_mouse_down(on_mouse_down2)
mcanvas1.on_mouse_down(on_mouse_down1)
mcanvas1.on_mouse_up(on_mouse_up1)

hbox = HBox(children = [mcanvas1, mcanvas2], layout={'width': '{}px'.format(2*SIZE)})

In [ ]:
# Initialisiere die Spielfelder

N = 9 # Spielfeldgroesse
players = ['You', 'Computer']
#Schiffe = Schiffslängen für das Spiel
Schiffe = [2,3,3,4,5]
Leben = [sum(Schiffe)] * 2 # [6, 6]

#Spielfeld für den Spieler für die eigenen Schiffe 
Spielfeld_Spieler = [[' '] * N for i in range(N)]
#Spielfeld für den Computer für die eigenen Schiffe 
Spielfeld_Computer = [[' '] * N for i in range(N)]
# um abhaenging vom Spieler ein Spielfeld waehlen zu koennen
Spielfelder = [Spielfeld_Spieler, Spielfeld_Computer]
Platzierung_Schiffe(Spielfeld_Computer)     

# Spieler beginnt mit dem Platzieren seiner Schiffe
mode='place'


# Audio Background Sound
audio_path = 'battleship_sound.mp3'


# Leinwand und log loeschen, grid und Spielfelder zeichnen
mcanvas1.clear()
mcanvas2.clear()
log.clear_output()

draw_grid(bg1)
draw_grid(bg2)
draw_spielfeld(fg1, Spielfeld_Spieler)

with log:
    print('Ships to place: {}. Place them on the left board with the mouse'.format(Schiffe).format(Schiffe))
    
display(hbox, log)  
Audio(audio_path, autoplay=True)